# Project Title: Data Lake Project
### Data Engineering Capstone Project

#### Project Summary
This project gathers the US I94 immigration data along with US airport code data and US city demographics data to create a data lake using Pyspark for future use.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import configparser
import os
import boto3
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
import pyspark.sql.functions as F
from pyspark.sql.functions import col

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

Create a data lake with Pyspark with the immigration, airport data and US city demographics by partitioning parquet files in table directories stored in AWS S3 for future use following schema-on-read semantics. The output parquet table directories could be used to analyze immigration trends focucing on immigration origins and airport destinations. The generated partitioned parquet files in table directories are:

- immigration_table
- city_table
- airport_table

Example of Future Use:

- Which US airports are the most traveled to in a year? Which ones are the most traveled to in a month for a particular year? What would it be the busiest monthly prediction for the upcoming years? What would it be the busiest days for the upcoming months? Such predictions could help out CBP (U.S. Customs and Border Protection) to forecast US airports insights by relying on advanced analytics.

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

I-94 Immigration Data: The SAS files come from US National Tourism and Trade Office and have a data dictionary file named "I94_SAS_Labels_Description.SAS". This project loads all immigration data for year 2016. All 12 files have more than 40 million records which fulfills the project requeriment to have a minimum of 1 million record in a file. This project also loads countries and visa categories from "I94_SAS_Labels_Description.SAS".

US City Demographic Data: This data is from OpenSoft and has US city demographics data. For more information, go to
https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/

Airport Code Data: This data is a list of US airport codes, US region, and other information related to aiport data.. For more information, go to https://datahub.io/core/airport-codes#data

In [2]:
# Read config file
config = configparser.ConfigParser()
config.read_file(open('capstone.cfg'))
os.environ['AWS_ACCESS_KEY_ID']=config['KEYS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['KEYS']['AWS_SECRET_ACCESS_KEY']

In [3]:
# Setup spark session 
spark = SparkSession.builder.\
appName("Capstone Project - US Immigration Data").\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11,org.apache.hadoop:hadoop-aws:2.7.0").\
config("spark.hadoop.fs.s3a.endpoint", "s3-us-west-2.amazonaws.com") .\
config("spark.hadoop.fs.s3a.awsAccessKeyId", os.environ['AWS_ACCESS_KEY_ID']).\
config("spark.hadoop.fs.s3a.awsSecretAccessKey", os.environ['AWS_SECRET_ACCESS_KEY']).\
config("spark.sql.broadcastTimeout", "36000").\
getOrCreate()

In [4]:
spark

In [5]:
OUTPUT_DATA = config['DATA']['OUTPUT_DATA']
BUCKET_NAME = config['DATA']['BUCKET_NAME']
OUTPUT_DIR = config['DATA']['OUTPUT_DIR']
I94_FOLDER = config['DATA']['I94_FOLDER']
print(I94_FOLDER)

../../data/18-83510-I94-Data-2016/


In [6]:
# Print all files under I94 data from year 2016
for filename in os.listdir(I94_FOLDER):
    print(filename)

i94_apr16_sub.sas7bdat
i94_sep16_sub.sas7bdat
i94_nov16_sub.sas7bdat
i94_mar16_sub.sas7bdat
i94_jun16_sub.sas7bdat
i94_aug16_sub.sas7bdat
i94_may16_sub.sas7bdat
i94_jan16_sub.sas7bdat
i94_oct16_sub.sas7bdat
i94_jul16_sub.sas7bdat
i94_feb16_sub.sas7bdat
i94_dec16_sub.sas7bdat


In [7]:
# Append function to merge dataframes
def append_dataframes(dfa,dfb):
    """
    Description: This function merges 2 dataframes
    Arguments: dataframe (a), dataframe (b)
    Returns: a new dataframe after union of 2 dataframes
    """
    lista = dfa.columns
    listb = dfb.columns
    for col in listb:
        if(col not in lista):
            dfa = dfa.withColumn(col, F.lit(None))
    for col in lista:
        if(col not in listb):
            dfb = dfb.withColumn(col, F.lit(None))
    return dfa.unionByName(dfb)

In [8]:
# Load all files under I94_FOLDER config variable
for f,filename in enumerate(os.listdir(I94_FOLDER)):
    if f == 0:
        df = spark.read.format('com.github.saurfang.sas.spark').load(I94_FOLDER + filename)
        df_i94 = df
    else:
         df = spark.read.format('com.github.saurfang.sas.spark').load(I94_FOLDER + filename)
         df_i94 = append_dataframes(df_i94,df) 

In [9]:
# Print total number of rows - it needs to be 40790529
print(df_i94.count())

40790529


In [10]:
# Print total number of columns
print(len(df_i94.columns))

34


In [11]:
# Print schema of df_i94 Spark dataframe
df_i94.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [12]:
# Display top 5 rows of df_i94 Spark dataframe
df_i94.show(5)

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+--------+-----------+-----------+----------+-----------+-------------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|validres|delete_days|delete_mexl|delete_dup|delete_visa|delete_recdup|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+--------+-----------+-----------+----------+-----------+-------------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|    

In [13]:
# Get US_CITY_DEMO config variable
US_CITY_DEMO = config['DATA']['US_CITY_DEMO']
print(US_CITY_DEMO)

./us-cities-demographics.csv


In [14]:
# Read the city demographics CSV file and print total number of rows and columns
df_city = spark.read.csv(US_CITY_DEMO, sep = ';', header=True)
print((df_city.count(), len(df_city.columns)))

(2891, 12)


In [15]:
# Display top 5 rows of df_city
df_city.show(5)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040| 

In [16]:
# Print schema of df_city
df_city.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



In [17]:
# Get AIRPORT_CODE config variable
AIRPORT_CODE = config['DATA']['AIRPORT_CODE']
print(AIRPORT_CODE)

./airport-codes_csv.csv


In [18]:
# Read the airport codes CSV file and print total number of rows and columns
df_airport = spark.read.csv(AIRPORT_CODE, header=True)
print((df_airport.count(), len(df_airport.columns)))

(55075, 12)


In [19]:
# Display top 5 rows of df_airport
df_airport.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

In [20]:
# Print schema of df_airport
df_airport.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [21]:
# Get I94_DATA_DICT config variable
I94_DATA_DICT = config['DATA']['I94_DATA_DICT']
print(I94_DATA_DICT)

./I94_SAS_Labels_Descriptions.SAS


In [22]:
# Read SAS labels and descriptions file then remove special characters
with open(I94_DATA_DICT) as f:
    lines = f.readlines()
lines = [line.replace('"','').replace('\n','').replace("'",'').replace(' ','').replace('\t','') for line in lines]

In [23]:
# Retrieve pair codes and names function from SAS labels and descriptions file
def get_value_pairs(value_pairs, pairs):
    """
    Description: This function retrieves the pair codes and names from SAS labels and descriptions file
    Arguments: Empty list of codes/names, lines to get codes/names from SAS file
    Returns: List of codes/names
    """
    for pair in pairs:
        value_code = pair.split("=")[0]
        value_name = pair.split("=")[1]
        value_pairs.append((value_code, value_name))
    return value_pairs

In [24]:
# Get list of all country codes and names from SAS labels and descriptions file
country_value_pairs = list()
countries = lines[10:245]
country_value_pairs = get_value_pairs(country_value_pairs, countries)
country_schema = StructType([
        StructField("country_code", StringType()),
        StructField("country_name", StringType())])
df_countries = spark.createDataFrame(data=country_value_pairs,schema=country_schema)
df_countries_new = df_countries.withColumn('country_code', df_countries['country_code'].cast(DoubleType()))
df_countries_new.printSchema()
df_countries_new.show(truncate=False)

root
 |-- country_code: double (nullable = true)
 |-- country_name: string (nullable = true)

+------------+---------------+
|country_code|country_name   |
+------------+---------------+
|236.0       |AFGHANISTAN    |
|101.0       |ALBANIA        |
|316.0       |ALGERIA        |
|102.0       |ANDORRA        |
|324.0       |ANGOLA         |
|529.0       |ANGUILLA       |
|518.0       |ANTIGUA-BARBUDA|
|687.0       |ARGENTINA      |
|151.0       |ARMENIA        |
|532.0       |ARUBA          |
|438.0       |AUSTRALIA      |
|103.0       |AUSTRIA        |
|152.0       |AZERBAIJAN     |
|512.0       |BAHAMAS        |
|298.0       |BAHRAIN        |
|274.0       |BANGLADESH     |
|513.0       |BARBADOS       |
|104.0       |BELGIUM        |
|581.0       |BELIZE         |
|386.0       |BENIN          |
+------------+---------------+
only showing top 20 rows



In [25]:
# Get list of all visa codes and names from SAS labels and descriptions file
visas = lines[1046:1049]
visa_value_pairs = list()
visa_value_pairs = get_value_pairs(visa_value_pairs, visas)
visa_schema = StructType([
        StructField("visa_code", StringType()),
        StructField("visa_type", StringType())])
df_visas = spark.createDataFrame(data=visa_value_pairs,schema=visa_schema)
df_visas_new = df_visas.withColumn('visa_code', df_visas['visa_code'].cast(DoubleType()))
df_visas_new.printSchema()
df_visas_new.show(truncate=False)

root
 |-- visa_code: double (nullable = true)
 |-- visa_type: string (nullable = true)

+---------+---------+
|visa_code|visa_type|
+---------+---------+
|1.0      |Business |
|2.0      |Pleasure |
|3.0      |Student  |
+---------+---------+



In [26]:
# Join df_countries and df_visas with i_94 
df_i94_new = df_i94.join(df_countries_new).where(df_i94['i94res'] == df_countries_new['country_code'])\
.join(df_visas_new).where(df_i94['i94visa'] == df_visas_new['visa_code'])

In [27]:
# Display top 5 records of consolidated i_94 Spark dataframe
df_i94_new.show(5)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+--------+-----------+-----------+----------+-----------+-------------+------------+------------+---------+---------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|         admnum|fltno|visatype|validres|delete_days|delete_mexl|delete_dup|delete_visa|delete_recdup|country_code|country_name|visa_code|visa_type|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+---------------+-----+--------+--------+-----------+-----------+----------+-----------+-------------+------------+-----

In [28]:
# Print total number of rows
print(df_i94_new.count())

38129404


In [29]:
# Print total number of columns
print(len(df_i94_new.columns))

38


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

- Remove duplicates from the 3 Spark dataframes: *immigration_table*, *city_table* and *airport_table*.
- Remove irrelevant columns from immigration_data: cicid, visapost, occup, dtadfile, entdepa, entdepd, dtaddto, validres, delete_days, delete_mexl, delete_dup, delete_visa and delete_recdup.
- Remove duplicate columns from immigration_data: "i94visa", "i94res".
- Replace all column names that have a space with underscore from city_table.
- Remove every row that has any null value from city_table and airport_table.

#### Cleaning Steps
Document steps necessary to clean the data

1. Create each of the 3 dataframe tables using distinct.
2. Remove columns that are not relevant from immigration_table.
3. Remove duplicate columns from immigration_data: "i94visa", "i94res".
4. Replace all column names that have a space with underscore from city_table.
5. Remove every row that has any null value from city_table and airport_table.

In [30]:
# Remove duplicate rows from the 3 Spark dataframes: immigration_table, city_table and airport_table
# along with columns that are not relevant
immigration_table = df_i94_new.select("i94yr", "i94mon", "i94cit","i94port", "arrdate", "i94mode", "i94addr", "depdate", "i94bir", "count","matflag","biryear","dtaddto","gender","insnum","airline","admnum","fltno","visatype","country_code","country_name","visa_code","visa_type").distinct()
# Print total number of rows and columns of immigration table
print(immigration_table.count(), len(immigration_table.columns))

38129329 23


In [31]:
# Print schema of immigration table
immigration_table.printSchema()

root
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- count: double (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)
 |-- country_code: double (nullable = true)
 |-- country_name: string (nullable = true)
 |-- visa_code: double (nullable = true)
 |-- visa_type: string (nullable = true)



In [32]:
# Remove duplicate rows of city table and replace all column names 
# that have a space with underscore from city_table. Drop any null values of city table.
city_table = df_city.select(col("City").alias("city"), col("State").alias("state"), col("Median Age").alias("median_age"), \
        col("Male Population").alias("male_population"), col("Female Population").alias("female_population"), \
        col("Total Population").alias("total_population"), col("Number of Veterans").alias("number_of_veterans"), \
        col("Foreign-born").alias("foreign_born"), col("Average Household Size").alias("average_household_size"), \
        col("State Code").alias("state_code"), col("Race").alias("race"), "count").na.drop().distinct()
# Print total number of rows and columns of city table 
print((city_table.count(), len(city_table.columns)))

(2875, 12)


In [33]:
# Print schema of city table
city_table.printSchema()

root
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- median_age: string (nullable = true)
 |-- male_population: string (nullable = true)
 |-- female_population: string (nullable = true)
 |-- total_population: string (nullable = true)
 |-- number_of_veterans: string (nullable = true)
 |-- foreign_born: string (nullable = true)
 |-- average_household_size: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- race: string (nullable = true)
 |-- count: string (nullable = true)



In [34]:
# Remove duplicate rows and drop any null values of airport table
airport_table = df_airport.na.drop().distinct()
# Print total number of rows and columns of airport table
print((airport_table.count(), len(airport_table.columns)))

(2746, 12)


In [35]:
# Print schema of airport dimension table
airport_table.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

Generate partitioned parquet files in table directories:

| Table Name      | Description        | Partition By  | 
| --- | --- | --- | 
| immigration_table | Fact table that has US i94 immigration data | i94year, i94mon and i94port |
| city_table | Dimmension table that has city demographics data | state_code |
| airport_table | Dimmension table that has US airport data | iata_code |

The immigration_table has been partitioned by year, month and airport code for better performance on aggregration queries. This could be even more helpful when loading i94 files from many years. The aiport_table has been aggregrated by iata_code (airport code) while city_table has been aggregrated by state_code. Data dictionary of data model is included below.

This data model was chosen since the project intention is to create a data lake with Pyspark with the immigration, airport and US city demographics data by partitioning parquet files in table directories stored in AWS S3 for future use following schema-on-read semantics.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Load all 12 I94 immigration files, city demographics file, airport code file, country information and visa data to Spark dataframes.
2. Join country and visa data to I94 immigration Spark dataframe.
3. Remove duplicated records for each of the 3 Spark dataframes: immigration_table, city_table and airport_table.
4. Remove columns that are not relevant from immigration_table.
5. Remove every row that has any null value from city_table and airport_table.
6. Create parquet table directories partitioned them by the columns listed above from the cleaned Spark dataframes.
7. Create a view for each of the cleaned Spark dataframes to execute SQL queries.
8. Execute quality checks on cleaned Spark dataframes and parquet table directories:
    - Check if each of the 3 Spark dataframes (immigration_table, city_table and airport_table) has records.
    - Check if each parquet directory exists in S3 bucket.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [36]:
# Create parquet file from immigration_table
immigration_table.write.partitionBy("i94yr","i94mon","i94port").mode("ignore").parquet(OUTPUT_DATA + "immigration_table.parquet")
#dataFrame.coalesce(1).write.format("parquet").mode("overwrite").save("temp.parquet")
# Create table view from immigration_table
immigration_table.createOrReplaceTempView("immigration_view")

In [37]:
# Create parquet file from city_table
city_table.write.partitionBy("state_code").mode("ignore").parquet(OUTPUT_DATA + "city_table.parquet")
# Create table view from city_table
city_table.createOrReplaceTempView("city_view")

In [38]:
# Create parquet file from airport_table
airport_table.write.partitionBy("iata_code").mode("ignore").parquet(OUTPUT_DATA + "airport_table.parquet")
# Create table view from airport_table
airport_table.createOrReplaceTempView("airport_view")

#### 4.2 Data Quality Checks

Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:

- Integrity constraints on the relational database (e.g., unique key, data type, etc.)
- Unit tests for the scripts to ensure they are doing the right thing
- Source/Count checks to ensure completeness

Run Quality Checks

- Check if each of the 3 Spark dataframes (immigration_table, city_table and airport_table) has records.
- Check if each parquet directory exists in S3 bucket.

In [39]:
## Data quality check function to confirm that each quality check has records
def quality_check(check_data,check_name,check_function):
    """
    Description: This function runs quality checks
    Arguments: Quality check, quality check name, check function
    Returns: Number of records for the quality check
    """
    
    if check_function == 'file_exist':    
        s3 = boto3.resource('s3')
        bucket = s3.Bucket(BUCKET_NAME)
        objs = list(bucket.objects.filter(Prefix=check_data))
        result = len(objs)
        if (len(objs) > 0):
            result = 'exist'
    elif check_function == 'count': 
        result = check_data.count() 
        
    if result == 0:
        print("FAILED DATA QUALITY CHECK for {} with zero records".format(check_name))
    else:
        print("PASSED DATA QUALITY CHECK for {} with {} records".format(check_name, result))

In [40]:
# Call quality check function to check that each of the 3 table dataframes has records
quality_check(immigration_table,'immigration_table','count')
quality_check(city_table,'city_table','count')
quality_check(airport_table,'airport_table','count')

PASSED DATA QUALITY CHECK for immigration_table with 38129329 records
PASSED DATA QUALITY CHECK for city_table with 2875 records
PASSED DATA QUALITY CHECK for airport_table with 2746 records


In [41]:
# Call quality check function to check that each parquet directory exists
parquet_dir_check = OUTPUT_DIR + 'immigration_table.parquet/'
quality_check(parquet_dir_check,'immigration_table.parquet','file_exist')
parquet_dir_check = OUTPUT_DIR + 'city_table.parquet/'
quality_check(parquet_dir_check,'city_table.parquet','file_exist')
parquet_dir_check = OUTPUT_DIR + 'airport_table.parquet/'
quality_check(parquet_dir_check,'airport_table.parquet','file_exist')

PASSED DATA QUALITY CHECK for immigration_table.parquet with exist records
PASSED DATA QUALITY CHECK for city_table.parquet with exist records
PASSED DATA QUALITY CHECK for airport_table.parquet with exist records


#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

- **immigration_table**: It came from 12 files related i94 immigration data residing in Udacity workspace with code and visa category data.

| Column Name | Description | 
| ----------- | ----------- |
| i94yr | 4 digit year |
| i94mon | 2 digit month |
| i94cit | 3 digit code of country in transit |
| ***i94port*** | 3 character code of aiport (***matches "aita_code" from airport_table***) |
| arrdate | arrival date |
| i94mode | 1 digit travel code |
| ***i94addr*** | 2 digit state code (***matches "state_code" from city_table***) |
| depdate | departure date |
| i94bir | age in years |
| count | count summary |
| matflag | match of arrival and departure |
| biryear | 4 digit of birth year |
| dtaddto | date to stay in US |
| gender | gender of the immigrant |
| insnum | immigration number |
| airline | airline code |
| admnjum | admission number |
| fltno | flight number |
| visatype | type of visa |
| country_code | 3 digit code of origin country |
| country_name | name of origin country |
| visa_code | 1 digit visa code |
| visa_code | visa category |

 - **city_table**: It came from US cities demographics file from Udacity workspace.

| Column Name | Description | 
| ----------- | ----------- |
| i94yr | 4 digit year |
| i94mon | 2 digit month |
| i94cit | 3 digit code of country in transit |
| ***i94port*** | 3 character code of aiport (***matches "aita_code" from airport_table***) |
| arrdate | arrival date |
| i94mode | 1 digit travel code |
| ***i94addr*** | 2 digit state code (***matches "state_code" from city_table***) |
| depdate | departure date |
| i94bir | age in years |
| count | count summary |
| matflag | match of arrival and departure |
| biryear | 4 digit of birth year |
| dtaddto | date to stay in US |
| gender | gender of the immigrant |
| insnum | immigration number |
| airline | airline code |
| admnjum | admission number |
| fltno | flight number |
| visatype | type of visa |
| country_code | 3 digit code of origin country |
| country_name | name of origin country |
| visa_code | 1 digit visa code |
| visa_code | visa category |
 
 | Column Name | Description | 
 | ----------- | ----------- |
 | city | name of city |
 | state | name of state |
 | median_age | median age of city |
 | male_population | population of males |
 | female_population | population of females |
 | total_population | total population of city |
 | number_of_veterans | number of veterans of city |
 | foreign_born | foreign born |
 | average_household_size | size of average household |
 | ***state_code*** | 2 digit state code (***matches "i94addr" from immigration_table***) |
 | race | race |
 | count | count |
 
 - **airport_table**: It came from US airport codes file from Udacity workspace.
 
 | Column Name | Description | 
 | ----------- | ----------- |
 | ident | airport identification |
 | type | airport type |
 | name | airport name | 
 | elevation_ft | feet elevation |
 | continent | continent |
 | iso_country | country code |
 | iso_region | 2 digit country - 2 digit state |
 | municipality | municipality |
 | gps code | GPS code |
 | ***iata_code*** | airport code (***matches "i94port" from immigration_table***) |
 | local_code | local code |
 | coordinates | coordinates |

| Column Name | Description | 
| ----------- | ----------- |
| i94yr | 4 digit year |
| i94mon | 2 digit month |
| i94cit | 3 digit code of country in transit |
| ***i94port*** | 3 character code of aiport (***matches "aita_code" from airport_table***) |
| arrdate | arrival date |
| i94mode | 1 digit travel code |
| ***i94addr*** | 2 digit state code (***matches "state_code" from city_table***) |
| depdate | departure date |
| i94bir | age in years |
| count | count summary |
| matflag | match of arrival and departure |
| biryear | 4 digit of birth year |
| dtaddto | date to stay in US |
| gender | gender of the immigrant |
| insnum | immigration number |
| airline | airline code |
| admnjum | admission number |
| fltno | flight number |
| visatype | type of visa |
| country_code | 3 digit code of origin country |
| country_name | name of origin country |
| visa_code | 1 digit visa code |
| visa_code | visa category |

#### Example of Future Use

- Which US airports are the most traveled to in a year? Which ones are the most traveled to in a month from a particular year? What would it be the busiest monthly prediction for the upcoming years? What would it be the busiest days for the upcoming months? Such predictions could help out CBP (U.S. Customs and Border Protection) to forecast US airports insights by relying on advanced analytics. 

In [42]:
# Top 10 busiest US airports in 2016
spark.sql("""
        SELECT 
            i.i94port AS port_code, 
            COUNT(*) AS immigrant_visits
        FROM immigration_view i 
        WHERE i.i94yr = 2016
        GROUP BY port_code
        ORDER BY immigrant_visits DESC
        LIMIT 10
""").show()

+---------+----------------+
|port_code|immigrant_visits|
+---------+----------------+
|      NYC|         6494862|
|      MIA|         4888188|
|      LOS|         4256277|
|      HHW|         2248676|
|      SFR|         2227517|
|      NEW|         1837420|
|      CHI|         1642329|
|      ORL|         1549953|
|      AGA|         1337858|
|      ATL|         1044287|
+---------+----------------+



In [43]:
# Top 20 busiest US airports in a month from 2016
spark.sql("""
        SELECT 
            i.i94mon AS month_code,
            i.i94port AS port_code,  
            COUNT(*) AS immigrant_visits
        FROM immigration_view i 
        WHERE i.i94yr = 2016
        GROUP BY month_code, port_code
        ORDER BY immigrant_visits DESC
        LIMIT 20
""").show()

+----------+---------+----------------+
|month_code|port_code|immigrant_visits|
+----------+---------+----------------+
|       7.0|      NYC|          711925|
|       8.0|      NYC|          682500|
|      10.0|      NYC|          644660|
|       9.0|      NYC|          620518|
|       5.0|      NYC|          607725|
|       6.0|      NYC|          603737|
|      12.0|      MIA|          541342|
|       3.0|      NYC|          508969|
|      12.0|      NYC|          480137|
|       4.0|      NYC|          473333|
|       7.0|      MIA|          460132|
|       7.0|      LOS|          457300|
|       8.0|      LOS|          439210|
|       9.0|      LOS|          433155|
|      11.0|      NYC|          427773|
|       8.0|      MIA|          421012|
|      10.0|      MIA|          420437|
|      11.0|      MIA|          418982|
|       3.0|      MIA|          415674|
|       1.0|      MIA|          404042|
+----------+---------+----------------+



#### Step 5: Complete Project Write Up
##### Clearly state the rationale for the choice of tools and technologies for the project.
- This project uses Python version 3.6.3 with Pyspark.
- Apache Spark (Pyspark) is used due to its parallelism and scalability when handling large datasets.
- Parquet output files in columnar format are used for aggregation.

##### Propose how often the data should be updated and why.
- The I94 data is aggregated on year, month and airport code basis. Therefore updating data on a monthly base is ideal.

##### Write a description of how you would approach the problem differently under the following scenarios:
##### The data was increased by 100x.
 - Use AWS EMR & Spark to process the data. This can be executed after creating a AWS EMR cluster and running capstone.py with the spark-submit command.
 
##### The data populates a dashboard that must be updated on a daily basis by 7am every day.
 - A schematized data warehouse should be created based on the parquet output files using Apache Airflow to schedule a Spark job to run on a daily basis. In case of any issue arises, email notification can be setup to notify a team to fix any errors. 
 
##### The database needed to be accessed by 100+ people.
 - Use AWS Redshift to load the parquet output files. Then a schematized data warehouse should be created based on the parquet output files. Apache Cassandra could also be used due to its scalability and good read performance for large datasets.